In [ ]:
!pip install --upgrade pip
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-g49e93gb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-g49e93gb
  Resolved https://github.com/huggingface/transformers.git to commit 831bc25d8fdb85768402f772cf65cc3d7872b211
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!gdown 1cCWH_eoa4Nq17XDHn6e1WIfHomdGWPKO

Downloading...
From (original): https://drive.google.com/uc?id=1cCWH_eoa4Nq17XDHn6e1WIfHomdGWPKO
From (redirected): https://drive.google.com/uc?id=1cCWH_eoa4Nq17XDHn6e1WIfHomdGWPKO&confirm=t&uuid=b70399bd-7130-4103-a180-c6e37d3b5612
To: /content/myaudio_tiny.tar.gz
100% 248M/248M [00:05<00:00, 42.7MB/s]


In [ ]:
!tar -xzvf /content/myaudio_tiny.tar.gz -C /content/

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import pandas as pd
import soundfile as sf


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!


In [ ]:
my_data = pd.read_excel('/content/myaudio_tiny/myaudio_tiny.xlsx')
path = '/content/myaudio_tiny/'
test_data = my_data.sample(30)

In [ ]:
whisper_texts = []
for i in range(len(test_data)):
    array, sampling_rate = sf.read(path +test_data.iloc[i]['audio'])
    sample = {'path':'/content/myaudio_tiny/myaudio/12200011.wav','array':array,'sampling_rate':sampling_rate}
    result = pipe(sample, generate_kwargs={"language": "persian"})
    whisper_texts.append(result["text"])

test_data['whisper_text'] = whisper_texts


In [ ]:
test_data.head()

,audio,text,whisper_text
638,myaudio/12441970.wav,هرچه از باقیمانده ی خوراکی ها مانده بود را بلع...,هرچی از باقی مانده ی خوراکی ها مانده بود را ب...
300,myaudio/12440015.wav,اهالی روستا آدم های محترمی بودند و وقت بی وقت...,احالی رستا آدم های محترم می بودند و وقت و بی ...
31,myaudio/450046.wav,دوم این که کیفیت بسیار بالایی ما در حروفچینی م...,دوم اینکه کیفیت بسیار بالایی ما در حروفچیدی م...
351,myaudio/12440129.wav,جویدن اش خیلی سخت نباشد و البته این به خاطر دن...,جویدنش خیلی سخت نباشد و البته این به خاطر دند...
249,myaudio/12220104.wav,کسی که اینو میخوره رفتارش رفتار انسان های سالم...,کسی که اینو میخوره رفتارش رفتار انسانهای سالم...
629,myaudio/12441946.wav,و رودخانه بیشتر شبیه یک جویبار باشد در چند صد ...,و رودخانه بیشتر شبیه یک جوی بار شد در چند ست ...
776,myaudio/12560031.wav,استان تهران اقلیمی متنوع دارد که این تنوع تحت ...,استان تهران اقلیمی متنوعه دارد که این تنوع تح...
586,myaudio/12441861.wav,کنار رودخانه اطراق کرد و صبح زود سفر آرام اش ر...,کنار رودخانه اتراخ کرد و صبح زود سفر آرامش را...
786,myaudio/12560041.wav,در اوایل خرداد هوا نسبتا گرم است گرم ترین ماه ...,در اوائل خرداد، هوا نسبتاً گرم است. گرمترین م...
804,myaudio/12560069.wav,این شهر به تدریج از طرف شمال توسعه یافت و مناز...,این شهر به تبریج از طرف شمال توصیه یافت و منا...


,audio,text,whisper_text
638,myaudio/12441970.wav,هرچه از باقیمانده ی خوراکی ها مانده بود را بلع...,هرچی از باقی مانده ی خوراکی ها مانده بود را ب...
300,myaudio/12440015.wav,اهالی روستا آدم های محترمی بودند و وقت بی وقت...,احالی رستا آدم های محترم می بودند و وقت و بی ...
31,myaudio/450046.wav,دوم این که کیفیت بسیار بالایی ما در حروفچینی م...,دوم اینکه کیفیت بسیار بالایی ما در حروفچیدی م...
351,myaudio/12440129.wav,جویدن اش خیلی سخت نباشد و البته این به خاطر دن...,جویدنش خیلی سخت نباشد و البته این به خاطر دند...
249,myaudio/12220104.wav,کسی که اینو میخوره رفتارش رفتار انسان های سالم...,کسی که اینو میخوره رفتارش رفتار انسانهای سالم...


In [ ]:
test_data.to_csv('test_data.csv')

In [ ]:
def process_dataset(voice_paths):
    whisper_texts = []
    for audio in voice_paths :
        array, sampling_rate = sf.read(audio)
        sample = {'path':audio,'array':array,'sampling_rate':sampling_rate}
        result = pipe(sample, generate_kwargs={"language": "persian"})
        whisper_texts.append(result["text"])
    return whisper_texts



SyntaxError: invalid syntax (<ipython-input-1-1b698f1a5c1c>, line 1)

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [ ]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.2 MB/s eta 0:00:00


In [ ]:
import faiss

In [ ]:
data = pd.read_csv("test_data.csv")

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
def search(query, texts):

    whisperembed = []
    for text in texts:
      inputswhisper = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
      outputs = model(**inputswhisper)
      embeddingswhisper = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
      whisperembed.append(embeddingswhisper)


    inputsquery = tokenizer(query, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputsquery)

    # Take the embeddings from the last layer
    embeddingsquery = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

    whisperembed_np = np.array(whisperembed)
    query_np = np.array([embeddingsquery])

    # Build an index with Faiss
    d = query_np.shape[1]  # Dimension of the vectors
    index = faiss.IndexFlatL2(d)
    index.add(whisperembed_np)

    # Search for the most similar texts
    k = len(texts)  # Number of texts to retrieve
    _, I = index.search(query_np, k)

    # Return the indices of the most similar texts
    return I[0].tolist()



In [ ]:
import numpy as np

result_indices = search(data['text'][2], data['whisper_text'])
print("Most similar texts indices:", result_indices)

Most similar texts indices: [2, 23, 5, 14, 18, 4, 10, 29, 27, 21, 19, 28, 22, 9, 0, 3, 11, 16, 6, 15, 26, 1, 24, 20, 12, 13, 7, 8, 17, 25]
